In [1]:
import pandas as pd
import requests
from urllib.parse import quote
import time

## import data
from \ 
https://www.oakville.ca/town-hall/elections/past-elections/2022-municipal-election/2022-municipal-election-results-by-location/'

In [2]:
# URL of the webpage containing the table
url = 'https://www.oakville.ca/town-hall/elections/past-elections/2022-municipal-election/2022-municipal-election-results-by-location/'


In [3]:
# Use pandas to read the HTML tables from the webpage
tables = pd.read_html(url)

# Display the number of tables found
print(f"Number of tables found: {len(tables)}")

# Assuming the first table is the one you're interested in
data = tables[0]

# Display the first few rows of the DataFrame
data.head()

Number of tables found: 18


,Location,Rob Burton,Julia Hanna,Jack Kukolic
0,0-1 Town Hall,429,402,54
1,1-1 Walton Memorial Church,659,763,44
2,1-2 Eastview Public School,415,507,39
3,1-3 Queen Elizabeth Park Community and Cultura...,531,541,55
4,1-4 Hope Bible Church,404,337,23


In [6]:
results_df = data.copy()

In [ ]:
# drop total row

results_df = results_df.iloc[:-1]

In [24]:
# clean the location to retrieve them in geofy

results_df['Location'] = results_df['Location'].str[4:]

results_df.head()


,Location,Rob Burton,Julia Hanna,Jack Kukolic
0,Hall,429,402,54
1,on Memorial Church,659,763,44
2,view Public School,415,507,39
3,n Elizabeth Park Community and Cultural Centre,531,541,55
4,Bible Church,404,337,23


## Retrieve and add the coordinates 

### use geopify api to get the coordinates via the reverse mode

In [9]:
# create list of vote_places

## extract unique value of vote places
vote_places = results_df['Location'].unique()

## create a list
vote_places_list = vote_places.tolist()

print(vote_places_list[0:5])
print(f'.............................................')
print(f'number of vote places : {len(vote_places_list)}')

['Town Hall', 'Walton Memorial Church', 'Eastview Public School', 'Queen Elizabeth Park Community and Cultural Centre', 'Hope Bible Church']
.............................................
number of vote places : 39


In [10]:
# API key geopify api

api_key = "8e0f597430e942f0ac38b55cf85ff32b"

In [11]:
# FOR LOOP can take 1m 50s for 75 places

# List to hold dictionaries for each place
places_coordinates = []  


# loop through the places in the list and retrive the ccordinates

for place in vote_places_list:

    ## complete the places to specify they are in oakville, ON

    

    full_place = f'{place}; Oakville; Ontario; Canada'
    
    try:
        encoded_place = quote(full_place)
        url = f"https://api.geoapify.com/v1/geocode/search?text={encoded_place}&apiKey={api_key}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if data.get('features'):
            latitude = data['features'][0]['properties']['lat']
            longitude = data['features'][0]['properties']['lon']

            place_entry = {
                'Location': place,
                'full_place' : full_place,
                'lon': longitude,
                'lat': latitude
            }
            places_coordinates.append(place_entry)

        else:
            place_entry = {
                'Location': place,
                'full_place' : full_place,
                'lon': 'none',
                'lat': 'none'
            }
            places_coordinates.append(place_entry)
            print(f"No coordinates found for {place}")

        time.sleep(0.5)

    except requests.RequestException as e:
        print(f"Request failed for {place}: {e}")

print(places_coordinates)

[{'Location': 'Town Hall', 'full_place': 'Town Hall; Oakville; Ontario; Canada', 'lon': -79.68764906171216, 'lat': 43.467343650000004}, {'Location': 'Walton Memorial Church', 'full_place': 'Walton Memorial Church; Oakville; Ontario; Canada', 'lon': -79.713292, 'lat': 43.393497}, {'Location': 'Eastview Public School', 'full_place': 'Eastview Public School; Oakville; Ontario; Canada', 'lon': -79.70952465379693, 'lat': 43.401299550000005}, {'Location': 'Queen Elizabeth Park Community and Cultural Centre', 'full_place': 'Queen Elizabeth Park Community and Cultural Centre; Oakville; Ontario; Canada', 'lon': -79.666672, 'lat': 43.447436}, {'Location': 'Hope Bible Church', 'full_place': 'Hope Bible Church; Oakville; Ontario; Canada', 'lon': -79.7405867658494, 'lat': 43.38512905}, {'Location': 'Palermo Public School', 'full_place': 'Palermo Public School; Oakville; Ontario; Canada', 'lon': -79.77605365397905, 'lat': 43.4320547}, {'Location': 'Sir John Colborne Centre for Seniors', 'full_place'

In [25]:
## create a DF with coordinates

coordinates_df = pd.DataFrame(places_coordinates)

coordinates_df.head()

,Location,full_place,lon,lat
0,Town Hall,Town Hall; Oakville; Ontario; Canada,-79.687649,43.467344
1,Walton Memorial Church,Walton Memorial Church; Oakville; Ontario; Canada,-79.713292,43.393497
2,Eastview Public School,Eastview Public School; Oakville; Ontario; Canada,-79.709525,43.401300
3,Queen Elizabeth Park Community and Cultural Ce...,Queen Elizabeth Park Community and Cultural Ce...,-79.666672,43.447436
4,Hope Bible Church,Hope Bible Church; Oakville; Ontario; Canada,-79.740587,43.385129


In [13]:
## export coordinates (in case we don't want to run the API again)
path = "Outputs/vote_places_coordinates_municipal.csv"
coordinates_df.to_csv(path,index=False)

## add coordinates to results df
to avoid looping again , use csv file generated in path

In [14]:
coordinates_df = pd.read_csv(path)
coordinates_df.head()


,Location,full_place,lon,lat
0,Town Hall,Town Hall; Oakville; Ontario; Canada,-79.687649,43.467344
1,Walton Memorial Church,Walton Memorial Church; Oakville; Ontario; Canada,-79.713292,43.393497
2,Eastview Public School,Eastview Public School; Oakville; Ontario; Canada,-79.709525,43.401300
3,Queen Elizabeth Park Community and Cultural Ce...,Queen Elizabeth Park Community and Cultural Ce...,-79.666672,43.447436
4,Hope Bible Church,Hope Bible Church; Oakville; Ontario; Canada,-79.740587,43.385129


In [ ]:
## add coordinates to resulats data frame (use merge )
results_wide = pd.merge(results_df, coordinates_df, how='left', left_on= 'Location', right_on='Location')

results_wide.head()

,Location,Rob Burton,Julia Hanna,Jack Kukolic,full_place,lon,lat
0,Town Hall,429,402,54,Town Hall; Oakville; Ontario; Canada,-79.687649,43.467344
1,Walton Memorial Church,659,763,44,Walton Memorial Church; Oakville; Ontario; Canada,-79.713292,43.393497
2,Eastview Public School,415,507,39,Eastview Public School; Oakville; Ontario; Canada,-79.709525,43.401300
3,Queen Elizabeth Park Community and Cultural Ce...,531,541,55,Queen Elizabeth Park Community and Cultural Ce...,-79.666672,43.447436
4,Hope Bible Church,404,337,23,Hope Bible Church; Oakville; Ontario; Canada,-79.740587,43.385129


In [16]:
### export to csv 

results_wide.to_csv("Outputs/results_municipal_wide.csv", index=False)

## convert to long format for other purposes

In [17]:
results_wide.columns

Index(['Location', 'Rob Burton', 'Julia Hanna', 'Jack Kukolic', 'full_place',
       'lon', 'lat'],
      dtype='object')

In [27]:
results_long = pd.melt(results_wide, id_vars=['Location', 'full_place', 'lon', 'lat' ], var_name='candidate', value_name='votes' )
results_long

,Location,full_place,lon,lat,candidate,votes
0,Town Hall,Town Hall; Oakville; Ontario; Canada,-79.687649,43.467344,Rob Burton,429
1,Walton Memorial Church,Walton Memorial Church; Oakville; Ontario; Canada,-79.713292,43.393497,Rob Burton,659
2,Eastview Public School,Eastview Public School; Oakville; Ontario; Canada,-79.709525,43.401300,Rob Burton,415
3,Queen Elizabeth Park Community and Cultural Ce...,Queen Elizabeth Park Community and Cultural Ce...,-79.666672,43.447436,Rob Burton,531
4,Hope Bible Church,Hope Bible Church; Oakville; Ontario; Canada,-79.740587,43.385129,Rob Burton,404
...,...,...,...,...,...,...
130,Glen Abbey Community Centre,Glen Abbey Community Centre; Oakville; Ontario...,-79.666672,43.447436,Jack Kukolic,34
131,Sixteen Mile Sports Complex,Sixteen Mile Sports Complex; Oakville; Ontario...,-79.749246,43.465963,Jack Kukolic,38
132,Oakville Trafalgar Community Centre,Oakville Trafalgar Community Centre; Oakville;...,-79.673749,43.453465,Jack Kukolic,15
133,Sheridan College,Sheridan College; Oakville; Ontario; Canada,-79.699213,43.469585,Jack Kukolic,2


In [28]:
# add rank to the result

# results_long['Rank'] = results_long.groupby(['Location', 'full_place', 'lon', 'lat']).cumcount() + 1
results_long['Rank'] = results_long.groupby(['Location', 'full_place', 'lon', 'lat'])['votes'].rank(ascending=False, method='dense')

results_long

,Location,full_place,lon,lat,candidate,votes,Rank
0,Town Hall,Town Hall; Oakville; Ontario; Canada,-79.687649,43.467344,Rob Burton,429,1.0
1,Walton Memorial Church,Walton Memorial Church; Oakville; Ontario; Canada,-79.713292,43.393497,Rob Burton,659,2.0
2,Eastview Public School,Eastview Public School; Oakville; Ontario; Canada,-79.709525,43.401300,Rob Burton,415,2.0
3,Queen Elizabeth Park Community and Cultural Ce...,Queen Elizabeth Park Community and Cultural Ce...,-79.666672,43.447436,Rob Burton,531,2.0
4,Hope Bible Church,Hope Bible Church; Oakville; Ontario; Canada,-79.740587,43.385129,Rob Burton,404,1.0
...,...,...,...,...,...,...,...
130,Glen Abbey Community Centre,Glen Abbey Community Centre; Oakville; Ontario...,-79.666672,43.447436,Jack Kukolic,34,6.0
131,Sixteen Mile Sports Complex,Sixteen Mile Sports Complex; Oakville; Ontario...,-79.749246,43.465963,Jack Kukolic,38,5.0
132,Oakville Trafalgar Community Centre,Oakville Trafalgar Community Centre; Oakville;...,-79.673749,43.453465,Jack Kukolic,15,6.0
133,Sheridan College,Sheridan College; Oakville; Ontario; Canada,-79.699213,43.469585,Jack Kukolic,2,3.0


In [29]:
results_long.to_csv("Outputs/results_municipal_long.csv", index=False)

In [ ]:
# CTRL

x = results_long.sort_values(['Location', 'Rank'])
x

,Location,full_place,lon,lat,candidate,votes,Rank
8,Brookdale Elementary School,Brookdale Elementary School; Oakville; Ontario...,-79.704576,43.424315,Rob Burton,263,1
53,Brookdale Elementary School,Brookdale Elementary School; Oakville; Ontario...,-79.704576,43.424315,Julia Hanna,297,2
98,Brookdale Elementary School,Brookdale Elementary School; Oakville; Ontario...,-79.704576,43.424315,Jack Kukolic,35,3
17,ClearView Christian Reformed Church,ClearView Christian Reformed Church; Oakville;...,-79.654973,43.491508,Rob Burton,243,1
62,ClearView Christian Reformed Church,ClearView Christian Reformed Church; Oakville;...,-79.654973,43.491508,Julia Hanna,191,2
...,...,...,...,...,...,...,...
66,West Oak Public School,West Oak Public School; Oakville; Ontario; Canada,-79.735381,43.448680,Julia Hanna,399,2
111,West Oak Public School,West Oak Public School; Oakville; Ontario; Canada,-79.735381,43.448680,Jack Kukolic,31,3
25,White Oaks Secondary School,White Oaks Secondary School; Oakville; Ontario...,-79.702205,43.461924,Rob Burton,244,1
70,White Oaks Secondary School,White Oaks Secondary School; Oakville; Ontario...,-79.702205,43.461924,Julia Hanna,324,2
